In [1]:
import dask.array as da
import cupy

from distributed import Client
from dask_hip import LocalHIPCluster
from time import time

In [2]:
def run_multi_gpu(device_array):
    t0 = time()
    (device_array + 1)[::2, ::2].sum().compute()
    t1 = time()
    print(f'{t1 - t0:.4f}')

In [3]:
def main():
    # prep random Dask array in **device** memory
    rs_d = da.random.RandomState(RandomState=cupy.random.RandomState)

    # actual allocation of Dask array on device memory
    x_d = rs_d.normal(10,
                      1,
                      size=(50_000, 50_000),
                      chunks=(1_000, 1_000))

    print('\nProcessing smaller cupy array...')
    with LocalHIPCluster(HIP_VISIBLE_DEVICES="0") as cluster:
        with Client(cluster) as client:
            run_multi_gpu(x_d)

    # bigger array
    x_d2 = rs_d.normal(10,
                       1,
                       size=(400_000, 400_000),    # array size 64X
                       chunks=(40_000, 40_000))    # total chunks: 10 x 10 = 100

    print('\nProcessing larger cupy array...')
    with LocalHIPCluster(HIP_VISIBLE_DEVICES="0") as cluster:
        with Client(cluster) as client:
            run_multi_gpu(x_d2)

    print('\nCompleted!')

In [4]:
main()


Processing smaller cupy array...


2023-09-22 20:38:50,424 - distributed.preloading - INFO - Creating preload: dask_hip.initialize
2023-09-22 20:38:50,424 - distributed.preloading - INFO - Import preload module: dask_hip.initialize


19.6551
device count inside worker init = 8

Processing larger cupy array...


2023-09-22 20:39:11,672 - distributed.preloading - INFO - Creating preload: dask_hip.initialize
2023-09-22 20:39:11,672 - distributed.preloading - INFO - Import preload module: dask_hip.initialize


64.3060
device count inside worker init = 8

Completed!


In [4]:
cluster = LocalHIPCluster(HIP_VISIBLE_DEVICES="0,1,2,3,4,5,6,7")

2023-09-22 21:29:44,128 - distributed.preloading - INFO - Creating preload: dask_hip.initialize
2023-09-22 21:29:44,128 - distributed.preloading - INFO - Import preload module: dask_hip.initialize
2023-09-22 21:29:44,129 - distributed.preloading - INFO - Creating preload: dask_hip.initialize
2023-09-22 21:29:44,129 - distributed.preloading - INFO - Import preload module: dask_hip.initialize
2023-09-22 21:29:44,138 - distributed.preloading - INFO - Creating preload: dask_hip.initialize
2023-09-22 21:29:44,139 - distributed.preloading - INFO - Import preload module: dask_hip.initialize
2023-09-22 21:29:44,149 - distributed.preloading - INFO - Creating preload: dask_hip.initialize
2023-09-22 21:29:44,150 - distributed.preloading - INFO - Import preload module: dask_hip.initialize
2023-09-22 21:29:44,150 - distributed.preloading - INFO - Creating preload: dask_hip.initialize
2023-09-22 21:29:44,150 - distributed.preloading - INFO - Import preload module: dask_hip.initialize
2023-09-22 21:2

In [5]:
cluster

LocalHIPCluster(97c394ef, 'tcp://127.0.0.1:39639', workers=8, threads=8, memory=503.72 GiB)

In [6]:
client = Client(cluster)

In [8]:
rs_d = da.random.RandomState(RandomState=cupy.random.RandomState)

In [9]:
x_d2 = rs_d.normal(10, 1, size=(400_000, 400_000), chunks=(40_000, 40_000))

In [12]:
run_multi_gpu(x_d2)

9.0685


In [13]:
client.shutdown()

device count inside worker init = 8
device count inside worker init = 8
device count inside worker init = 8
device count inside worker init = 8
device count inside worker init = 8
device count inside worker init = 8
device count inside worker init = 8
device count inside worker init = 8


In [14]:
cluster.close()